In [7]:
from __future__ import annotations

from numbers import Number
from typing import List



class Matrix(object):
    ''' A simple naive matrix class

    Members
    -------
    _A: List[List[Number]]
        The list of rows that store all the matrix values

    Parameters
    ----------
    A: List[List[Number]]
        The list of rows that store all the matrix values
    clone_matrix: Optional[bool]
        A flag to require a full copy of `A`'s data structure.

    Raises
    ------
    ValueError
        If there are two lists having a different number of values
    '''
    def __init__(self, A: List[List[Number]], clone_matrix: bool = True):
        num_of_cols = None

        for i, row in enumerate(A):
            if num_of_cols is not None:
                if num_of_cols != len(row):
                    raise ValueError('This is not a matrix')
            else:
                num_of_cols = len(row)

        if clone_matrix:
            self._A = [[value for value in row] for row in A]
        else:
            self._A = A

    @property
    def num_of_rows(self) -> int:
        return len(self._A)

    @property
    def num_of_cols(self) -> int:
        if len(self._A) == 0:
            return 0

        return len(self._A[0])

    def copy(self):
        A = [[value for value in row] for row in self._A]

        return Matrix(A, clone_matrix=False)

    def __getitem__(self, y: int):
        ''' Return one of the rows

        Parameters
        ----------
        y: int
            the index of the rows to be returned

        Returns
        -------
        List[Number]
            The `y`-th row of the matrix
        '''
        return self._A[y]

    def __iadd__(self, A: Matrix) -> Matrix:
        ''' Sum a matrix to this matrix and update it

        Parameters
        ----------
        A: Matrix
            The matrix to be summed up

        Returns
        -------
        Matrix
            The matrix corresponding to the sum between this matrix and
            that passed as parameter

        Raises
        ------
        ValueError
            If the two matrices have different sizes
        '''

        if (self.num_of_cols != A.num_of_cols or
                self.num_of_rows != A.num_of_rows):
            raise ValueError('The two matrices have different sizes')

        for y in range(self.num_of_rows):
            for x in range(self.num_of_cols):
                self[y][x] += A[y][x]

        return self

    def __add__(self, A: Matrix) -> Matrix:
        ''' Sum a matrix to this matrix

        Parameters
        ----------
        A: Matrix
            The matrix to be summed up

        Returns
        -------
        Matrix
            The matrix corresponding to the sum between this matrix and
            that passed as parameter

        Raises
        ------
        ValueError
            If the two matrices have different sizes
        '''
        res = self.copy()

        res += A

        return res

    def __isub__(self, A: Matrix) -> Matrix:
        ''' Subtract a matrix to this matrix and update it

        Parameters
        ----------
        A: Matrix
            The matrix to be subtracted up

        Returns
        -------
        Matrix
            The matrix corresponding to the subtraction between this matrix and
            that passed as parameter

        Raises
        ------
        ValueError
            If the two matrices have different sizes
        '''

        if (self.num_of_cols != A.num_of_cols or
                self.num_of_rows != A.num_of_rows):
            raise ValueError('The two matrices have different sizes')

        for y in range(self.num_of_rows):
            for x in range(self.num_of_cols):
                self[y][x] -= A[y][x]

        return self

    def __sub__(self, A: Matrix) -> Matrix:
        ''' Subtract a matrix to this matrix

        Parameters
        ----------
        A: Matrix
            The matrix to be subtracted up

        Returns
        -------
        Matrix
            The matrix corresponding to the subtraction between this matrix and
            that passed as parameter

        Raises
        ------
        ValueError
            If the two matrices have different sizes
        '''
        res = self.copy()

        res -= A

        return res

    def __mul__(self, A: Matrix) -> Matrix:
        ''' Multiply one matrix to this matrix

        Parameters
        ----------
        A: Matrix
            The matrix which multiplies this matrix

        Returns
        -------
        Matrix
            The row-column multiplication between this matrix and that passed
            as parameter

        Raises
        ------
        ValueError
            If the number of columns of this matrix is different from the
            number of rows of `A`
        '''
        return gauss_matrix_mult(self, A)

    def __rmul__(self, value: Number) -> Matrix:
        ''' Multiply one matrix by a numeric value

        Parameters
        ----------
        value: Number
            The numeric value which multiplies this matrix

        Returns
        -------
        Matrix
            The multiplication between `value` and this matrix

        Raises
        ------
        ValueError
            If `value` is not a number
        '''

        if not isinstance(value, Number):
            raise ValueError('{} is not a number'.format(value))

        return Matrix([[value*elem for elem in row] for row in self._A],
                      clone_matrix=False)

    def submatrix(self, from_row: int, num_of_rows: int,
                  from_col: int, num_of_cols: int) -> Matrix:
        ''' Return a submatrix of this matrix

        Parameters
        ----------
        from_row: int
            The first row to be included in the submatrix to be returned
        num_of_rows: int
            The number of rows to be included in the submatrix to be returned
        from_col: int
            The first col to be included in the submatrix to be returned
        num_of_cols: int
            The number of cols to be included in the submatrix to be returned

        Returns
        -------
        Matrix
            A submatrix of this matrix
        '''
        A = [row[from_col:from_col+num_of_cols]
             for row in self._A[from_row:from_row+num_of_rows]]

        return Matrix(A, clone_matrix=False)

    def assign_submatrix(self, from_row: int, from_col: int, A: Matrix, sum: bool = False):
        for y, row in enumerate(A):
            self_row = self[y + from_row]
            for x, value in enumerate(row):
                if sum==True:
                    self_row[x + from_col] += value
                else:
                    self_row[x + from_col] = value

    def __repr__(self):
        return '\n'.join('{}'.format(row) for row in self._A)


class IdentityMatrix(Matrix):
    ''' A class for identity matrices

    Parameters
    ----------
    size: int
        The size of the identity matrix
    '''
    def __init__(self, size: int):
        A = [[1 if x == y else 0 for x in range(size)]
             for y in range(size)]

        super().__init__(A)



## 1) Strassen Algorithm for $ 2^N $x $ 2^N $ matrices

In [8]:


def gauss_matrix_mult(A: Matrix, B: Matrix) -> Matrix:
    ''' Multiply two matrices by using Gauss's algorithm

    Parameters
    ----------
    A: Matrix
        The first matrix to be multiplied
    B: Matrix
        The second matrix to be multiplied

    Returns
    -------
    Matrix
        The row-column multiplication of the matrices passed as parameters

    Raises
    ------
    ValueError
        If the number of columns of `A` is different from the number of
        rows of `B`
    '''
    if A.num_of_cols != B.num_of_rows:
        raise ValueError("The matrices can not be multiplied")

    result = [[0 for col in range(B.num_of_cols)] for row in range(A.num_of_rows)]

    for row in range(A.num_of_rows):
        for col in range(B.num_of_cols):
            value = 0
            for k in range(A.num_of_cols):
                value += A[row][k]*B[k][col]
            result[row][col] = value
    return Matrix(result, clone_matrix=False)


def get_matrix_quadrant(A: Matrix) -> tuple(Matrix, Matrix, Matrix, Matrix):

    A11 = A.submatrix(from_row=0 , num_of_rows=A.num_of_rows//2 , from_col=0, num_of_cols=A.num_of_cols//2)
    A12 = A.submatrix(from_row=0 , num_of_rows=A.num_of_rows//2 , from_col=A.num_of_cols//2, num_of_cols=A.num_of_cols//2)
    A21 = A.submatrix(from_row=A.num_of_rows//2 , num_of_rows=A.num_of_rows//2 , from_col=0, num_of_cols=A.num_of_cols//2)
    A22 = A.submatrix(from_row=A.num_of_rows//2 , num_of_rows=A.num_of_rows//2 , from_col=A.num_of_cols//2, num_of_cols=A.num_of_cols//2)

    return (A11,A12,A21,A22)


def strassen_matrix_mult(A: Matrix, B: Matrix) -> Matrix:
    if A.num_of_cols != B.num_of_rows:
        raise ValueError("The matrices can not be multiplied")
    
    if max(A.num_of_rows,A.num_of_cols,B.num_of_cols) < 32:
        return gauss_matrix_mult(A,B)
    
    A11, A12, A21, A22 = get_matrix_quadrant(A)
    B11, B12, B21, B22 = get_matrix_quadrant(B)
    
    S1 = B12 - B22
    S2 = A11 + A12
    S3 = A21 + A22
    S4 = B21 - B11
    S5 = A11 + A22
    S6 = B11 + B22
    S7 = A12 - A22
    S8 = B21 + B22
    S9 = A11 - A21
    S10 = B11 + B12
    
    P1 = strassen_matrix_mult(A11,S1)
    P2 = strassen_matrix_mult(S2,B22)
    P3 = strassen_matrix_mult(S3,B11)
    P4 = strassen_matrix_mult(A22,S4)
    P5 = strassen_matrix_mult(S5,S6)
    P6 = strassen_matrix_mult(S7,S8)
    P7 = strassen_matrix_mult(S9,S10)
    
    C11 = P5 + P4 - P2 + P6
    C12 = P1 + P2
    C21 = P3 + P4
    C22 = P5 + P1 - P3 - P7
    
    result = Matrix([[0 for col in range(B.num_of_cols)] for row in range(A.num_of_rows)], clone_matrix=False)
    
    result.assign_submatrix(from_row=0, from_col=0 , A = C11)
    result.assign_submatrix(from_row=0, from_col=A.num_of_cols//2 , A = C12)
    result.assign_submatrix(from_row=A.num_of_rows//2, from_col=0,A = C21)
    result.assign_submatrix(from_row=A.num_of_rows//2, from_col=A.num_of_cols//2, A = C22)
    
    return result
    

## 2) Generalized Strassen Algorithm 

In order to generalize the `strassen_matrix_mult` function to deal with square matrices of any dimension $A \in \mathbb{R}^{m \times m}$ and $B \in \mathbb{R}^{m \times m}$ where m is not a power of two. We can calculate $\hat n = 2^{\left \lceil{log_2(m)}\right \rceil }$ which is the smallest integer power of 2 greater than m. Further on we can instatiate two new square matrices $A_{aux},B_{aux} \in \mathbb{R}^{\hat n \times \hat n}$ allocating in their top-left corner respectively the $A$ and $B$ matrix and filling all the remaning rows and columns with 0. Thus we can call the original function `strassen_matrix_mult` on those two new matrices $A_{aux}$ and $B_{aux}$ obtaining as a result a matrix $C_{aux} \in \mathbb{R}^{\hat n \times \hat n}$ which has in his top-left corner a matrix $C \in \mathbb{R}^{m \times m}$ and all the other rows and columns filled with zeros. Eventually, we simply have to extract the submatrix $C$ from $C_{aux}$ and there we have our result.
This technique require many allocation, nevertheless we can show that the asymptotic complexity does not change. Indeed we must show that $ T(\hat n) \in \Theta(m^{log_2(7)})$, namely the complexity for the multiplication of the two new matrices with dimension $\hat n \times \hat n$ belong to $\Theta(m^{log_2(7)})$.

We've seen that that $T(\hat n) \in \Theta(\hat n^{log_2(7)})$ where $\hat n = 2^{\left \lceil{log_2(m)}\right \rceil }$,

 we can state that $ 2^{\left \lceil{log_2(m)}\right \rceil } < 2^{log_2(m) + 1}$ 
 
 which imply that $\hat n < 2^{log_2(m)+1} = 2 \times m$. 
 
 This means that $\hat n^{log_2(7)} < 2\times m^{log_2(7)}$ and so $\hat n^{log_2(7)} \in O(m^{log_2(7)})$,
 
 still we know that $\hat n > m$ from which comes that $\hat n^{log_2(7)} \in \Omega( m^{log_2(7)})$, 
 
 thus $\hat n^{log_2(7)} \in \Theta(m^{log_2(7)})$ which proves our thesis. 
 
 Indeed since $\Theta(\hat n^{log_2(7)}) = \Theta(m^{log_2(7)})$ we know that $T(\hat n) \in \Theta(m^{log_2(7)})$.

All this reasoning could be extent to a more general case where $A \in \mathbb{R}^{n \times k}$ and $B \in \mathbb{R}^{k \times p}$ are two rectangular matrices, in this case infact we can take $m$ as the $max(n,k,p)$ and again $\hat n = 2^{\left \lceil{log_2(m)}\right \rceil }$ which means that all the demonstration would be in function of the maximal dimension of the problem, yet again we would obtain the same thesis.


Regarding the implementation itself I've procedeed in this way: I create an auxiliary function `strassen_matrix_mult_generic` to which I pass $A$ and $B$, firstly I check that the two matrices are compatible and then I check if they're already square and with dimension power of 2, if this is the case I simply return the `strassen_matrix_mult` on $A$ and $B$, otherwise I perform the algorithm exposed above, call the `strassen_matrix_mult` on $A_{aux}$ an $B_{aux}$ and finally return the submatrix $C$.

In [9]:

import math

def strassen_matrix_mult_generic(A: Matrix, B: Matrix) -> Matrix:
    if A.num_of_cols != B.num_of_rows:
        raise ValueError("The matrices can not be multiplied")

    tmp = A.num_of_cols*A.num_of_rows*B.num_of_cols
    if ((math.log(tmp,2).is_integer()) & (math.log(tmp,2) == 3*math.log(A.num_of_cols,2))):
        return strassen_matrix_mult(A,B)
    else:
        n = 2**math.ceil(math.log(max(A.num_of_rows,A.num_of_cols,B.num_of_cols),2))

    A_aux = Matrix([[0 for x in range(n)] for y in range(n)])
    B_aux = Matrix([[0 for x in range(n)] for y in range(n)])
   
    A_aux.assign_submatrix(0,0,A)
    B_aux.assign_submatrix(0,0,B)
            
    
    C_aux = strassen_matrix_mult(A_aux,B_aux)

    C = C_aux.submatrix(from_row=0,num_of_rows=A.num_of_rows, from_col=0,num_of_cols=B.num_of_cols)

    return C

## 3) 
In order to reduce the number of auxiliary matrices firstable we can avoid to allocate all the matrices $S$ which contains the sum of the submatrices and directly perform the summation on the recursive call. Moreover we can allocate all the matrices $P_i$ in the same space, indeed we can use them once at a time; in order to do this I modified the function `assign_submatrix` to take an extra parameter, for which, if it is true, the submatrix will be added. In this way the total number of auxiliary matrices used will be 9. I have also implemented another version where I get rid of the allocation of the submatrices $A_{i,j}$ and $B_{i,j}$, my aspectation was that the time saved avoiding the allocation of the submatrices would be exceeded or at least compensated by the additional computational time deriving from extracting many time the same submatrices.
Below are the measurment that I took, my expectation has partially been confirmed, infact the second version perform very similar to the first one (sometimes better sometime worse), however they both performed aroud a $5\%$ better than the original version which I think it's a good result.

In [10]:
def strassen_matrix_mult_mem(A: Matrix, B: Matrix) -> Matrix:
    
    if max(A.num_of_rows,A.num_of_cols,B.num_of_cols) < 32:
        return gauss_matrix_mult(A,B)
    
    A11, A12, A21, A22 = get_matrix_quadrant(A)
    B11, B12, B21, B22 = get_matrix_quadrant(B)

    result = Matrix([[0 for col in range(B.num_of_cols)] for row in range(A.num_of_rows)], clone_matrix = False)
    
    P = strassen_matrix_mult_mem(A11,B12 - B22)
    result.assign_submatrix(0,result.num_of_cols//2 , P , sum =True)
    result.assign_submatrix(result.num_of_rows//2, result.num_of_cols//2, P - strassen_matrix_mult_mem(A11 - A21,B11 + B12), sum =True)
    P = strassen_matrix_mult_mem(A11 + A12,B22)
    result.assign_submatrix(0, 0 ,strassen_matrix_mult_mem(A12 - A22,B21 + B22)-P, sum =True)
    result.assign_submatrix(0, result.num_of_cols//2 ,  P, sum =True)
    P = strassen_matrix_mult_mem(A21 + A22,B11)
    result.assign_submatrix(result.num_of_rows//2, 0,P, sum =True)
    result.assign_submatrix(result.num_of_rows//2, result.num_of_cols//2,  -1*P, sum =True)
    P = strassen_matrix_mult_mem(A22,B21 - B11)
    result.assign_submatrix(result.num_of_rows//2, 0, P, sum =True)
    result.assign_submatrix(0, 0 , P, sum =True)
    P = strassen_matrix_mult_mem(A11 + A22,B11 + B22)
    result.assign_submatrix(0, 0 , P, sum =True)
    result.assign_submatrix(result.num_of_rows//2, result.num_of_cols//2, P, sum =True)
    
    
    
    return result
    

In [11]:
def strassen_matrix_mult_mem_2(A: Matrix, B: Matrix) -> Matrix:
    
    if max(A.num_of_rows,A.num_of_cols,B.num_of_cols) < 32:
        return gauss_matrix_mult(A,B)

    result = Matrix([[0 for col in range(B.num_of_cols)] for row in range(A.num_of_rows)], clone_matrix = False)
    
    P = strassen_matrix_mult_mem(A.submatrix(from_row=0 , num_of_rows=A.num_of_rows//2 , 
    from_col=0, num_of_cols=A.num_of_cols//2), B.submatrix(from_row=0 , num_of_rows=A.num_of_rows//2 , from_col=A.num_of_cols//2, num_of_cols=A.num_of_cols//2) - B.submatrix(from_row=A.num_of_rows//2 ,num_of_rows=A.num_of_rows//2 , from_col=A.num_of_cols//2, num_of_cols=A.num_of_cols//2))
    result.assign_submatrix(0,result.num_of_cols//2 , P , sum =True)
    result.assign_submatrix(result.num_of_rows//2, result.num_of_cols//2, P - strassen_matrix_mult_mem(A.submatrix(from_row=0 , num_of_rows=A.num_of_rows//2 , from_col=0, num_of_cols=A.num_of_cols//2) - A.submatrix(from_row=A.num_of_rows//2 , num_of_rows=A.num_of_rows//2 , from_col=0, num_of_cols=A.num_of_cols//2),B.submatrix(from_row=0 , num_of_rows=A.num_of_rows//2 , from_col=0, num_of_cols=A.num_of_cols//2) + B.submatrix(from_row=0 , num_of_rows=A.num_of_rows//2 , from_col=A.num_of_cols//2, num_of_cols=A.num_of_cols//2)), sum =True)
    P = strassen_matrix_mult_mem(A.submatrix(from_row=0 , num_of_rows=A.num_of_rows//2 , from_col=0, num_of_cols=A.num_of_cols//2) + A.submatrix(from_row=0 , num_of_rows=A.num_of_rows//2 , from_col=A.num_of_cols//2, num_of_cols=A.num_of_cols//2),B.submatrix(from_row=A.num_of_rows//2 , num_of_rows=A.num_of_rows//2 , from_col=A.num_of_cols//2, num_of_cols=A.num_of_cols//2))
    result.assign_submatrix(0, 0 ,strassen_matrix_mult_mem(A.submatrix(from_row=0 , num_of_rows=A.num_of_rows//2 , from_col=A.num_of_cols//2, num_of_cols=A.num_of_cols//2) - A.submatrix(from_row=A.num_of_rows//2 , num_of_rows=A.num_of_rows//2 , from_col=A.num_of_cols//2, num_of_cols=A.num_of_cols//2),B.submatrix(from_row=A.num_of_rows//2 , num_of_rows=A.num_of_rows//2 , from_col=0, num_of_cols=A.num_of_cols//2) + B.submatrix(from_row=A.num_of_rows//2 , num_of_rows=A.num_of_rows//2 , from_col=A.num_of_cols//2, num_of_cols=A.num_of_cols//2))-P, sum =True)
    result.assign_submatrix(0, result.num_of_cols//2 ,  P, sum =True)
    P = strassen_matrix_mult_mem(A.submatrix(from_row=A.num_of_rows//2 , num_of_rows=A.num_of_rows//2 , from_col=0, num_of_cols=A.num_of_cols//2) + A.submatrix(from_row=A.num_of_rows//2 , num_of_rows=A.num_of_rows//2 , from_col=A.num_of_cols//2, num_of_cols=A.num_of_cols//2),B.submatrix(from_row=0 , num_of_rows=A.num_of_rows//2 , from_col=0, num_of_cols=A.num_of_cols//2))
    result.assign_submatrix(result.num_of_rows//2, 0,P, sum =True)
    result.assign_submatrix(result.num_of_rows//2, result.num_of_cols//2,  -1*P, sum =True)
    P = strassen_matrix_mult_mem(A.submatrix(from_row=A.num_of_rows//2 , num_of_rows=A.num_of_rows//2 , from_col=A.num_of_cols//2, num_of_cols=A.num_of_cols//2),B.submatrix(from_row=A.num_of_rows//2 , num_of_rows=A.num_of_rows//2 , from_col=0, num_of_cols=A.num_of_cols//2) - B.submatrix(from_row=0 , num_of_rows=A.num_of_rows//2 , from_col=0, num_of_cols=A.num_of_cols//2))
    result.assign_submatrix(result.num_of_rows//2, 0, P, sum =True)
    result.assign_submatrix(0, 0 , P, sum =True)
    P = strassen_matrix_mult_mem(A.submatrix(from_row=0 , num_of_rows=A.num_of_rows//2 , from_col=0, num_of_cols=A.num_of_cols//2) + A.submatrix(from_row=A.num_of_rows//2 , num_of_rows=A.num_of_rows//2 , from_col=A.num_of_cols//2, num_of_cols=A.num_of_cols//2),B.submatrix(from_row=0 , num_of_rows=A.num_of_rows//2 , from_col=0, num_of_cols=A.num_of_cols//2) + B.submatrix(from_row=A.num_of_rows//2 , num_of_rows=A.num_of_rows//2 , from_col=A.num_of_cols//2, num_of_cols=A.num_of_cols//2))
    result.assign_submatrix(0, 0 , P, sum =True)
    result.assign_submatrix(result.num_of_rows//2, result.num_of_cols//2, P, sum =True)
    
    
    
    return result

In [12]:
from sys import stdout
from random import random
from timeit import timeit
for i in range(11):
    size = 2**i
    stdout.write(f'{size}')
    A = Matrix([[random() for x in range(size)] for y in range(size)])
    B = Matrix([[random() for x in range(size)] for y in range(size)])
    for funct in ['strassen_matrix_mult', 'strassen_matrix_mult_mem', 'strassen_matrix_mult_mem_2']:
        T = timeit(f'{funct}(A,B)', globals=locals(), number=1)
        stdout.write('\t{:.3f}'.format(T)) 
        stdout.flush()
    stdout.write('\n')

1	0.000	0.000	0.000
2	0.000	0.000	0.000
4	0.000	0.000	0.000
8	0.000	0.000	0.000
16	0.001	0.001	0.001
32	0.008	0.009	0.009
64	0.051	0.047	0.048
128	0.362	0.347	0.340
256	2.551	2.428	2.427
512	18.051	17.207	17.194
1024	128.321	122.343	122.632


## 4)

If we consider the function `strassen_matrix_mult_mem_2`, which I believe is the implementation of the algorithm which require the minimum amount of auxiliary space; to evaluate the algorithm we need he space to allocate the matrix $A$, the matrix $B$, the result and finally the space for the recursive call. Starting from the beginnig: we need $3\times n^2$ space for the $A$, $B$ and the result, then we will allocate $3 \times (\frac{n}{2})^2$ for the recursive call (it will allocate again a matrix $A$, a $B$ and  a result but this time of dimension $\frac{n}{2}$), this process will be done recursively until the end of the recursive tree will be reached. We must consider only $log_2(n)$ recursive calls and not the total number, indeed only $log_2(n)$ calls will live on the stack at the same time.

If we call $S(n)$ the space complexity of the function we can say that $S(n) = 3n^2 + S(\frac{n}{2})$ from which comes that:

$S(n) = \sum_{i=0}^{log_2(n)} 3(\frac{n}{2^i})^2$ = $4n^2-1$

If we consider that the minimum space required to perform the gauss algorithm is $3n^2$ than the auxiliary space required by strassen is $n^2-1$.


To calculate the actual number of bytes needed we just have to multiply the number of bytes occupied by an element of the matrix for $n^2-1$. 